In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
# Hacer la petición base
r = requests.get('https://molinerosenlinea.com/tienda/')

In [3]:
r.status_code

200

In [5]:
soup = BeautifulSoup(r.text, 'html.parser')

In [17]:
# Items list
products_lists = soup.select('.products-grid ol li div span a')
products_lists = [x.get('href') for x in products_lists]
products_lists

['https://molinerosenlinea.com/producto/eq-sonido-lg-rm1-speaker-lg/',
 'https://molinerosenlinea.com/producto/eq-sonido-lg-cm4360-lg/',
 'https://molinerosenlinea.com/producto/mueble-p-micro-oregon-indum/',
 'https://molinerosenlinea.com/producto/div-c-entret-montana-indum/',
 'https://molinerosenlinea.com/producto/div-c-entret-sevilla-indum/',
 'https://molinerosenlinea.com/producto/eq-sonido-lg-cj44-440watts-lg/',
 'https://molinerosenlinea.com/producto/gavetero-clasico-5g-25x48-indum/',
 'https://molinerosenlinea.com/producto/barra-d-sonido-lg-sl4-lg/',
 'https://molinerosenlinea.com/producto/cama-unip-foam-relax-pop-fam/',
 'https://molinerosenlinea.com/producto/estufa-mabe-20-ema5105ab0-bco-mbe/',
 'https://molinerosenlinea.com/producto/div-c-entret-bolton-indum/',
 'https://molinerosenlinea.com/producto/eq-sonido-lg-cj45-770watts-lg/',
 'https://molinerosenlinea.com/producto/jgo-comedor-4s-pekin-c-nogal-indum/',
 'https://molinerosenlinea.com/producto/eq-sonido-lg-rn5-speaker-lg

In [18]:
# Detail Product
d = requests.get(products_lists[0])
d.status_code

200

In [19]:
s = BeautifulSoup(d.text, 'html.parser')

In [22]:
# Title Product
title = s.find('h1').get_text().strip().lower()
title

'eq. sonido lg rm1 speaker * lg'

In [25]:
# Brand
brand = s.find('div', attrs={'class':'product-item-brandname'}).get_text().lower()
brand

'lg'

In [38]:
# Real price
real_price = s.find('span', attrs={'class':'old-price'}).find('span', attrs={'class':'price'}).get_text().replace('L', '').replace(',','').strip()
real_price

'3999'

In [41]:
# Actual Price
actual_price = s.find('span', attrs={'class':'special-price'}).find('span', attrs={'class':'price'}).get_text().replace('L', '').replace(',','').strip()
actual_price

'3200'

In [54]:
item = {
    'title': title,
    'brand': brand,
    'real_price':convert_lempiras_to_dollars(real_price),
    'actual_price': convert_lempiras_to_dollars(actual_price),
    'url': d.url
}
item

{'title': 'eq. sonido lg rm1 speaker * lg',
 'brand': 'lg',
 'real_price': 167.96,
 'actual_price': 134.4,
 'url': 'https://molinerosenlinea.com/producto/eq-sonido-lg-rm1-speaker-lg/'}

# Creación de funciones auxiliares

In [67]:
# Lempiras to Dollars

def convert_lempiras_to_dollars(price:str) -> float:
    """Convert lempiras to dollars using actual price"""
    
    return round(int(price) * 0.042, 2)

In [90]:
def get_all_products(request) -> list:
    """From page take all products"""
    
    try:
        r = requests.get(request)
    except:
        print(f'Hubo un error en la url {request}')
        return None
    
    if r.status_code == 200:
        print(f'Scrapeando la categoria en url {request}')
        soup = BeautifulSoup(r.text, 'html.parser')
        
        products_lists = soup.select('.products-grid ol li div span a')
        products_lists = [x.get('href') for x in products_lists]
        
        return products_lists
    else:
        print(f'Codigo HTTP {r.status_code}. Se saltó a la siguiente ejecución')
        return None

In [91]:
def get_products_detail(request) -> dict:
    """Get a product URL and return a dict of the item"""
    
    try:
        r = requests.get(request)
    except:
        print(f'Hubo un error en la url {request}')
        return None
    
    if r.status_code == 200:
        print(f'Scrapeando el producto en la url {request}')
        s = BeautifulSoup(r.text, 'html.parser')
        
        # Title
        
        title = s.find('h1').get_text().strip().lower()
        
        # Brand
        brand = s.find('div', attrs={'class':'product-item-brandname'}).get_text().lower()
        
        # Real Price
        real_price = s.find('span', attrs={'class':'old-price'}).find('span', attrs={'class':'price'}).get_text().replace('L', '').replace(',','').strip()
        
        # Actual Price
        actual_price = s.find('span', attrs={'class':'special-price'}).find('span', attrs={'class':'price'}).get_text().replace('L', '').replace(',','').strip()
        
        item = {
            'title': title,
            'brand': brand,
            'real_price':convert_lempiras_to_dollars(real_price),
            'actual_price': convert_lempiras_to_dollars(actual_price),
            'url': d.url
        }
        
        return item
    else:
        print(f'Codigo HTTP {r.status_code}. Se saltó a la siguiente ejecución')
        return None

# Probando Funciones Auxiliares

In [68]:
details_products = []

for product in products_lists:
    details_products.append(get_products_detail(product))

In [70]:
len(details_products)

16

# Creando el Scraper

In [92]:
all_products = []

try:
    base = requests.get('https://molinerosenlinea.com/tienda/')
except:
    print('La URL base dio error')
    
if base.status_code == 200:

    sopa = BeautifulSoup(base.text, 'html.parser')

    pages = sopa.select('.pages ul li:last-child a span:last-child')[0].get_text()

    for i in range(1, int(pages) + 1):
        
        for product in get_all_products(f'https://molinerosenlinea.com/tienda/page/{i}/'):
            all_products.append(get_products_detail(product))
                                        
else:
    print(f'Codigo HTTP {base.status_code}. ERROR')
        

Scrapeando la categoria en url https://molinerosenlinea.com/tienda/page/1/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/eq-sonido-lg-rm1-speaker-lg/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/eq-sonido-lg-cm4360-lg/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/mueble-p-micro-oregon-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/div-c-entret-montana-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/div-c-entret-sevilla-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/eq-sonido-lg-cj44-440watts-lg/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/gavetero-clasico-5g-25x48-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/barra-d-sonido-lg-sl4-lg/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/cama-unip-foam-relax-pop-fam/
Scrapeando el producto en la

Scrapeando el producto en la url https://molinerosenlinea.com/producto/cama-queen-olympia-extra-sense-plus-dc/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/jgo-sala-sofia-3-pzas-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/recamara-king-girasol-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/aire-aco-mabe-mmi12cdbwccax8-12000-btu/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/cama-king-olympia-extra-sense-dco/
Scrapeando la categoria en url https://molinerosenlinea.com/tienda/page/6/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/lavadora-mabe-19kg-lma79113vbab0-mbe/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/estufa-mabe-30-cma3020fxo-inox-mbe/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/jgo-sala-catracho-3-pzas-indum/
Scrapeando el producto en la url https://molinerosenlinea.com/producto/lavador

In [94]:
len(all_products)

115